In [ ]:
movie = Movie.objects.all()[0]

In [51]:
MovieUserScore.objects.filter(movie_id=8)[0].score

10.0

In [52]:
mus = MovieUserScore()

In [61]:
len(MovieUserScore.objects.filter(movie_id=8, user_id=1))

1

In [64]:
len(MovieUserScore.objects.filter(movie_id=1, user_id=1))

0

In [26]:
mus = MovieUserScore()

In [19]:
mus.movie = Movie.objects.all()[0]

In [30]:
mus.user_id

In [21]:
mus.movie_id

In [60]:
from rest_framework.authtoken.models import Token

In [67]:
Token.objects.get(key='663e11d8cec1fac1b67df0229149952be998eb17').user_id

4

In [14]:
import os
import django
import requests
import datetime
import json
from collections import OrderedDict

from tqdm import tqdm
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [134]:
URL = 'https://api.themoviedb.org/3'
API_KEY = '164acb58532a315bea423c96031d8a71'

In [146]:
url_ = f'{URL}/movie/{499028}/videos?api_key=164acb58532a315bea423c96031d8a71'

## 추천점수

In [6]:
Keyword.objects.all()[1].word

'비올때'

In [46]:
keywords = ['반전', '비올때', '혼자보기 좋은', '넷플릭스']
movie_dict = {}
for keyword in keywords:
    kw_object = Keyword.objects.get(word=keyword)
    movies = kw_object.movies.all()
    movie_ids = [x.id for x in movies]
    recommend_list = KM.objects.filter(movie__in=movie_ids, word=kw_object.pk)
    # recommend_list.order_by('-score')[:20]

    for rl in recommend_list:
        mv = rl.movie
        if movie_dict.get(mv.id):
            movie_dict[mv.id] += rl.score
        else:
            movie_dict[mv.id] = rl.score
top20 = list(OrderedDict(sorted(movie_dict.items(), key=lambda t:-t[1])).items())[:20]
data = []

In [9]:
import numpy as np

def mm(arr):
    mm_result = (arr - min(min(arr), 2)) / (max(arr) - min(min(arr), 2))
    other = [x-int(x) for x in arr]
    return mm_result, other

result = mm(np.array([x[1] for x in top20]))
result = [round(x, 2) for x in result[0] * 9 + result[1]]

data = []
for i in range(len(top20)):
    data.append((top20[i][0], result[i]))

In [ ]:
for i in top20:
    mv_id = i[0]
    score = i[1]
    mv = Movie.objects.get(pk=mv_id)
    movie_data = {
        'id': mv.pk,
        'title_ko': mv.title_ko,
        'poster_path': mv.poster_path,
        'score': score
        }
    data.append(movie_data)

result = {'data':  data}